# Model training Diabetes Prediction Challenge

## 1. Imports 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import balanced_accuracy_score
from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

import optuna

c:\VScode\Python\MachineLearning\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Helper functions

In [2]:
def create_my_balanced_accuracy(weights):
    """
    Creates scoring function for cross_val_score with weights 
    """
    def my_scoring(est,X,y):
        preds = est.predict(X)
        wagi = np.asarray([weights[i] for i in y])

        score = balanced_accuracy_score(y,preds,sample_weight=wagi)

        return score
    
    return my_scoring


In [3]:
def calculate_weights(y: pd.Series):
    """
    Calculates weights for inbalanced class datasets.
    Returns the dictionary with the weights for each class.

    Args:
        y: The target variable
    """
    values = y.value_counts()
    weights = {}
    total = len(y)

    for class_label, count in values.items():
        other_counts = total - count
        weights[class_label] = 1.0 / other_counts if other_counts > 0 else 0

    return weights
        

In [4]:
def create_transformer(scaler_name, imputer_strategy,quantitative_columns,categorical_columns):
    if scaler_name == "StandardScaler":
        scaler = StandardScaler()
    elif scaler_name == "RobustScaler":
        scaler = RobustScaler()
    elif scaler_name == "MinMaxScaler":
        scaler = MinMaxScaler()

    numeric_transformer = Pipeline([
        ('inputer', SimpleImputer(strategy=imputer_strategy)),
        ('scaler', scaler)
    ])

    categorical_transformer = Pipeline([
        ('imputer',SimpleImputer(strategy='constant',fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, quantitative_columns),
            ('cat', categorical_transformer, categorical_columns)
        ],
        remainder='passthrough'
    )

    return preprocessor

In [5]:
def create_best_model(X_train,y_train,weights,quantitative_columns,categorical_columns):
    skf = StratifiedKFold(n_splits=5,shuffle=True)

    def objective(trial:optuna.trial.Trial):
        n_features = trial.suggest_int('n_features',3,X_train.shape[1])
        model_name = trial.suggest_categorical('model_name',["SVC","KNN"])
        estimator_name = trial.suggest_categorical("estimator_name",["SVC","GBC"])

        if model_name == "SVC":
            degree = 3
            C = trial.suggest_float("C", 0.2, 50)
            kernel = trial.suggest_categorical("kernel", ["rbf","poly","linear"])

            if kernel == "poly":
                degree = trial.suggest_int("degree",2,5)

            model = SVC(C=C, kernel=kernel, degree=degree)
        elif model_name == "KNN":
            n_neighbors = trial.suggest_int("n_neighbors", 3, 16)
            weights_knn = trial.suggest_categorical("weights_knn",["uniform","distance"])

            model = KNeighborsClassifier(n_neighbors=n_neighbors,weights=weights_knn)

        preprocessor_name = trial.suggest_categorical("preprocessor",["StandardScaler","RobustScaler","MinMaxScaler"])
        imputer_strategy = trial.suggest_categorical("imputer_strategy",['median',"mean"])

        preprocessor = create_transformer(preprocessor_name,imputer_strategy,quantitative_columns,categorical_columns)

        if estimator_name == "SVC":
            estimator = SVC(kernel="linear")
        elif estimator_name == "GBC":
            estimator = GradientBoostingClassifier()

        pipe = Pipeline([
            ("Preprocessor", preprocessor),
            ("RFE",RFE(estimator=estimator,n_features_to_select=n_features)),
            ("Classifier",model)
        ])

        scorer = create_my_balanced_accuracy(weights)

        scores = cross_val_score(
            pipe,
            X_train,
            y_train,
            cv=skf,
            scoring=scorer,
            n_jobs=-1
        )

        return scores.mean()

    study = optuna.create_study(direction="maximize")
    study.optimize(objective,n_trials=30)

    return study.best_params, study.best_value


## 3. Training

In [6]:
train_data = pd.read_csv("../Data/DiabetesPredictionChallenge/train.csv")
test_data = pd.read_csv("../Data/DiabetesPredictionChallenge/test.csv")

In [7]:
X_train = train_data.drop(columns=["id","diagnosed_diabetes"],axis=1)
y_train = train_data['diagnosed_diabetes']

X_test = test_data.drop(columns=["id"],axis=1)

In [8]:
X_train.isna().sum()

age                                   0
alcohol_consumption_per_week          0
physical_activity_minutes_per_week    0
diet_score                            0
sleep_hours_per_day                   0
screen_time_hours_per_day             0
bmi                                   0
waist_to_hip_ratio                    0
systolic_bp                           0
diastolic_bp                          0
heart_rate                            0
cholesterol_total                     0
hdl_cholesterol                       0
ldl_cholesterol                       0
triglycerides                         0
gender                                0
ethnicity                             0
education_level                       0
income_level                          0
smoking_status                        0
employment_status                     0
family_history_diabetes               0
hypertension_history                  0
cardiovascular_history                0
dtype: int64

In [9]:
categorical_columns = ["gender","ethnicity","education_level","income_level","smoking_status","employment_status","family_history_diabetes","hypertension_history","cardiovascular_history"]
quantitative_columns = [name for name in train_data.columns if name not in categorical_columns and name not in ["id","diagnosed_diabetes"]]

In [10]:
weights = calculate_weights(y_train)
best_params,best_score = create_best_model(X_train,y_train,weights,quantitative_columns,categorical_columns)

[I 2025-12-10 19:22:41,309] A new study created in memory with name: no-name-1309408d-cac7-47b4-806c-b1f9733e4378
[W 2025-12-10 19:26:22,287] Trial 0 failed with parameters: {'n_features': 19, 'model_name': 'KNN', 'estimator_name': 'GBC', 'n_neighbors': 5, 'weights_knn': 'uniform', 'preprocessor': 'StandardScaler', 'imputer_strategy': 'median'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\VScode\Python\MachineLearning\venv\Lib\site-packages\optuna\study\_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\posze\AppData\Local\Temp\ipykernel_25672\1119777995.py", line 42, in objective
    scores = cross_val_score(
             ^^^^^^^^^^^^^^^^
  File "c:\VScode\Python\MachineLearning\venv\Lib\site-packages\sklearn\utils\_param_validation.py", line 218, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\VScode\Python\MachineLearn

KeyboardInterrupt: 

In [ ]:
if best_params['estimator_name'] == "SVC":
    estimator = SVC(kernel='linear')
elif best_params['estimator_name'] == "GBC":
    estimator = GradientBoostingClassifier()

preprocessor = create_transformer(best_params['preprocessor_name'],best_params['imputer_strategy'],quantitative_columns,categorical_columns)

if best_params['model_name'] == "KNN":
    model = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'],weights=best_params['weights_knn'])
elif best_params['model_name'] == "SVC":
    model = SVC(C=best_params["C"],kernel=best_params['kernel'],degree=best_params['degree'])

pipe = Pipeline([
    ("Preprocessor",preprocessor),
    ("RFE",RFE(estimator=estimator,n_features_to_select=best_params['n_features'])),
    ('Classifier', model)
])

pipe.fit(X_train,y_train)